Project Ideas

- Google Maps Distance API - https://developers.google.com/maps/documentation/distance-matrix/intro
    - Set a marker A, set marker B, return the travel time
    - Set a marker A, calculate routes to multiple points on circle of radius X
    - Set a marker A, calculate routes to many points on a grid within circle of radius x
    
- House price API
    - Find average price at point A
    
- Display / Output
    - Load a google map
    - Display points A, B
    - Display a circle with many B points
    - Display a circle with many B points inside
    - Display a colour coded point by house price etc
    
    

# Super Basic Example!

In [139]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
from urllib.request import urlretrieve
from urllib.parse import urlencode
import tqdm

#!pip install polyline

import polyline

API_KEY = open('API_key.txt','r').read()

Origin = 'Seattle'
Destination = 'Washington DC'
travel_mode = 'transit'

google_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins=' + Origin + '&destinations=' + Destination + '&key=' + API_KEY + '&mode=' +travel_mode 
google_request = requests.get(google_url)

google_json = google_request.json()
print(json.dumps(google_json, indent=2))

{
  "destination_addresses": [
    "Washington, DC, USA"
  ],
  "origin_addresses": [
    "Seattle, WA, USA"
  ],
  "rows": [
    {
      "elements": [
        {
          "distance": {
            "text": "4,713 km",
            "value": 4712627
          },
          "duration": {
            "text": "2 days 17 hours",
            "value": 235500
          },
          "status": "OK"
        }
      ]
    }
  ],
  "status": "OK"
}


In [197]:

travel_mode = 'walking'
# driving, walking, bicycling, transit   are the possible options
from urllib.request import urlretrieve
from urllib.parse import urlencode


def distance_api(start_lat, start_lon, travel_mode, end_lat, end_lon, rebuild_table):
    
    global logging_table
    global Origin
    global Destination
    
    #read in API key from local directory
    API_KEY = open('API_key.txt','r').read()
    
    Origin = str(str(start_lat) + ', ' + str(start_lon))
    Destination = str(str(end_lat) + ', ' + str(end_lon))
    
    #convert coordinates to encoded polyline to make url easier to parse
    origin_polyline = 'enc:' + polyline.encode([(start_lat, start_lon)], 5) + ':'
    destination_polyline = 'enc:' + polyline.encode([(end_lat, end_lon)], 5) + ':'
    
    
    google_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins=' + origin_polyline + '&destinations=' + destination_polyline + '&key=' + API_KEY + '&mode=' +travel_mode 
    
    google_request = requests.get(google_url)

    google_json = google_request.json()

    distance_text = google_json['rows'][0]['elements'][0]['distance']['text']
    duration_text = google_json['rows'][0]['elements'][0]['duration']['text']

   # print(Origin + ' is ' + distance_text + ' away from ' + Destination +'. It will take ' + duration_text + ' to get there by ' + travel_mode)

    result_list = [datetime.datetime.now(),Origin,Destination,distance_text,duration_text,travel_mode]

    #initialise logging table
    
    #print('rebuild_table = ' + str(rebuild_table))
    if rebuild_table == 1:
        del logging_table
    
    try:
        logging_table
    except NameError:
        #print('nameerror flagged')
        column_labels = ['runtime','Origin','Destination','distance','travel_time','travel_mode']
        logging_table = pd.DataFrame(columns = column_labels)
        logging_table = logging_table.append(pd.Series(result_list, index = logging_table.columns), ignore_index=True)
    else:
        #print('no nameerror')
        logging_table = logging_table.append(pd.Series(result_list, index = logging_table.columns), ignore_index=True)
      
    return (logging_table)

In [160]:
distance_api(start_lat= 51.520406,start_lon= -0.085969, travel_mode= 'transit', end_lat= 51.640425, end_lon= 0.4384, rebuild_table = False)

,runtime,Origin,Destination,distance,travel_time,travel_mode,point_no
0,2020-03-03 18:56:10.613924,"51.520406, -0.085969","51.39999215917177, 0.23865764315821344",42.7 km,1 hour 15 mins,transit,1
1,2020-03-03 18:56:15.337877,"51.520406, -0.085969","51.700070099067375, -0.085969",35.3 km,1 hour 47 mins,transit,1
2,2020-03-03 18:56:15.532847,"51.520406, -0.085969","51.69612861621194, 0.004853592904968709",32.1 km,1 hour 18 mins,transit,1
3,2020-03-03 18:56:15.696163,"51.520406, -0.085969","51.68447843625486, 0.09170638197054494",30.2 km,1 hour 7 mins,transit,1
4,2020-03-03 18:56:15.918244,"51.520406, -0.085969","51.6656343958041, 0.17079322354235316",34.4 km,1 hour 30 mins,transit,1
5,2020-03-03 18:56:16.137417,"51.520406, -0.085969","51.64042840337344, 0.23865764315821344",39.0 km,2 hours 13 mins,transit,1
6,2020-03-03 18:56:16.335411,"51.520406, -0.085969","51.60997162129962, 0.2923339000528647",30.8 km,58 mins,transit,1
7,2020-03-03 18:56:16.540671,"51.520406, -0.085969","51.57560423812722, 0.32947650493176556",33.9 km,1 hour 15 mins,transit,1
8,2020-03-03 18:56:16.719262,"51.520406, -0.085969","51.53883532020751, 0.3484625684154494",43.4 km,1 hour 52 mins,transit,1
9,2020-03-03 18:56:16.962994,"51.520406, -0.085969","51.501275645809564, 0.3484625684154494",39.1 km,1 hour 45 mins,transit,1


In [229]:
import math

# Monzo Office Location
start_lat = 51.520406
start_lon = -0.085969


def circle_coords(start_lat,start_lon,distance,n_points):
    
    global coordinates_df
    
    R = 6378.1 #Radius of the Earth
 
    
    lat1 = math.radians(start_lat) #Current lat point converted to radians
    lon1 = math.radians(start_lon) #Current long point converted to radians
    
    # create a dataframe to populate
    column_labels = ['i','bearing','distance','start_lat','start_lon','end_lat','end_lon']
    coordinates_df = pd.DataFrame(columns = column_labels)
    
    for i in range(0, n_points+1):
                
        brng = math.radians(i * (360/n_points))    #convert bearing to radians
        lat1 = math.radians(start_lat) #Current lat point converted to radians
        lon1 = math.radians(start_lon) #Current long point converted to radians
        
        end_lat = math.degrees(math.asin( math.sin(lat1)*math.cos(distance/R) + math.cos(lat1)*math.sin(distance/R)*math.cos((brng))))
        end_lon = math.degrees(lon1 + math.atan2(math.sin(brng)*math.sin(distance/R)*math.cos(lat1), math.cos(distance/R)-math.sin(lat1)*math.sin(lat2)))
        
        coords_list = [i+1, i*(360/n_points), distance, start_lat, start_lon, end_lat, end_lon]
        
        coordinates_df = coordinates_df.append(pd.Series(coords_list, index = coordinates_df.columns), ignore_index=True)

    return (coordinates_df)


In [208]:
circle_coords(start_lat = 51.520406, start_lon = -0.085969, distance = 20, n_points = 30)

,i,bearing,distance,start_lat,start_lon,end_lat,end_lon
0,1.0,0.0,20.0,51.520406,-0.085969,51.700070,-0.085969
1,2.0,12.0,20.0,51.520406,-0.085969,51.696129,0.004854
2,3.0,24.0,20.0,51.520406,-0.085969,51.684478,0.091706
3,4.0,36.0,20.0,51.520406,-0.085969,51.665634,0.170793
4,5.0,48.0,20.0,51.520406,-0.085969,51.640428,0.238658
5,6.0,60.0,20.0,51.520406,-0.085969,51.609972,0.292334
6,7.0,72.0,20.0,51.520406,-0.085969,51.575604,0.329477
7,8.0,84.0,20.0,51.520406,-0.085969,51.538835,0.348463
8,9.0,96.0,20.0,51.520406,-0.085969,51.501276,0.348463
9,10.0,108.0,20.0,51.520406,-0.085969,51.464567,0.329477


In [232]:
# try and combine the functions!

#circle_coords(start_lat = 51.520406, start_lon = -0.085969, distance = 20, n_points = 30)
#distance_api(start_lat= 51.520406,start_lon= -0.085969, travel_mode= 'transit', end_lat= 51.640425, end_lon= 0.4384, rebuild_table = False)



def circle_distance(start_latitude, start_longitude, number_points):
    
    #this flag makes sure the logging table gets cleared
    flag = True
    
    circle_coords(start_lat = start_latitude, start_lon = start_longitude, distance = 10, n_points = number_points)
    
    for i in range(1,number_points+2):
        coordinates_df_filtered = coordinates_df[coordinates_df['i'] == i]
        distance_api(start_lat = coordinates_df_filtered.iloc[0]['start_lat'], start_lon= coordinates_df_filtered.iloc[0]['start_lon'], travel_mode = 'transit', end_lat=coordinates_df_filtered.iloc[0]['end_lat'], end_lon= coordinates_df_filtered.iloc[0]['end_lon'], rebuild_table = i)
        
        #print('Point number: ' + str(coordinates_df_filtered.iloc[0]['i']) +' done!')
    return (logging_table)

    
    
    

In [233]:
circle_distance(start_latitude= 51.520406,start_longitude = -0.085969,number_points= 24)



,runtime,Origin,Destination,distance,travel_time,travel_mode
0,2020-03-03 20:09:35.994427,"51.520406, -0.085969","51.610238049533685, -0.085969",15.5 km,56 mins,transit
1,2020-03-03 20:09:36.212099,"51.520406, -0.085969","51.607171148035015, -0.02943942499688691",15.8 km,54 mins,transit
2,2020-03-03 20:09:36.377401,"51.520406, -0.085969","51.59818064166191, 0.02323765607881572",15.2 km,40 mins,transit
3,2020-03-03 20:09:36.553520,"51.520406, -0.085969","51.583882477661874, 0.0684723471052802",16.1 km,42 mins,transit
4,2020-03-03 20:09:36.747798,"51.520406, -0.085969","51.56525549714392, 0.10318201874378949",14.8 km,32 mins,transit
5,2020-03-03 20:09:36.936712,"51.520406, -0.085969","51.54357353149911, 0.12500136104245368",16.9 km,42 mins,transit
6,2020-03-03 20:09:37.128017,"51.520406, -0.085969","51.520317402433555, 0.13244351869057022",18.6 km,1 hour 3 mins,transit
7,2020-03-03 20:09:37.378966,"51.520406, -0.085969","51.49707314311691, 0.12500136104245368",19.4 km,1 hour 3 mins,transit
8,2020-03-03 20:09:37.587370,"51.520406, -0.085969","51.47542360628357, 0.10318201874378954",19.0 km,1 hour 8 mins,transit
9,2020-03-03 20:09:37.820432,"51.520406, -0.085969","51.456840924474555, 0.06847234710528022",16.6 km,49 mins,transit


In [234]:
# next step is to figure out how to plot a set of points onto a map...

# import gmplot package 

#!pip install gmplot
import gmplot 
  
latitude_list = coordinates_df['end_lat'] 
longitude_list =coordinates_df['end_lon'] 
  
gmap3 = gmplot.GoogleMapPlotter(51.520406, 
                                -0.085969, 13 )  
  
# scatter method of map object  
# scatter points on the google map 
gmap3.scatter( latitude_list, longitude_list, '# FF0000', 
                              size = 40, marker = False ) 
  
# Plot method Draw a line in 
# between given coordinates 
gmap3.plot(latitude_list, longitude_list,  
           'cornflowerblue', edge_width = 2.5) 
  
gmap3.draw('./map_out_2.html')



In [235]:
#found this code here
# https://towardsdatascience.com/heat-map-your-google-payments-with-python-54ed110854d

#!pip install folium

import pandas as pd
import folium
import os
import sys
from folium.plugins import HeatMap

def main():
    # Read map data from CSV
    with open(sys.argv[1], 'r', encoding='utf8') as file:
        map_data = pd.read_csv(file)

    # Find highest purchase amount
    max_amount = float(map_data['amount'].max())

    # Makes a new map centered on the given location and zoom
    startingLocation = [-19.2, 146.8] # EDIT THIS WITH YOUR CITIES GPS COORDINATES!
    hmap = folium.Map(location=startingLocation, zoom_start=7)

    # Creates a heatmap element
    hm_wide = HeatMap( list(zip(map_data.lat.values, map_data.lon.values, map_data.amount.values)),
                        min_opacity=0.2,
                        max_val=max_amount,
                        radius=17, blur=15,
                        max_zoom=1)

    # Adds the heatmap element to the map
    hmap.add_child(hm_wide)

    # Saves the map to heatmap.hmtl
    hmap.save(os.path.join('.', 'heatmap.html'))
    
    if __name__ == "__main__":
    main()

     |████████████████████████████████| 91 kB 2.8 MB/s eta 0:00:011
